# Welcome to the Covid Tracking Example!

## Workshop Steps

Now that you have opened up the MyBinder environment and are reading this, you are already on the right track! Inside this environment, you will also find:

* sample scripts: This is a folder containing the base of the scripts that you will be working with to finish the exercise. Please look for the triple exclamation points (!!!) as that means that you are being asked to write some code to get things to work!
* helpers: This contains a script that you may need to put inside your data job's folder prior to executing, if you want to reset the data job's properties. You may want to do such a thing if you want to ingest the entirety of the data again - not recommended.
* README.md: This is just the README file you saw on the Github page.
* requirements.txt: This is a list of the required libraries that were installed upon startup.
* setup.ipynb: The file you are reading right now! Think of this as your home page.
* Other system files - postBuild and start: No need to worry about these. They are needed for the setup with our DB.

## Step 0: Set up SQL Magic

Let's set up the notebook to work with SQL using the code blocks below:

In [ ]:
%config SqlMagic.autocommit=False

In [ ]:
%load_ext sql

You can browse the various different tables in the default schema.

In [ ]:
%sql SHOW TABLES FROM default

You can query a table in the following way. Please note that we will want you to name the tables with a suffix containing your last name, your sports team, and your favorite drink. For example, instead of the table being called "covid_cases_europe_daily", you need to rename it to something like "covid_cases_europe_daily_smith_basel_pepsi".

Using the code cell below, you can run any query against the created tables. Please use the example below:

In [ ]:
%sql select count(*) from mysql.default.covid_cases_europe_daily

You can also drop tables, but please be very careful with this. Only do it when you are sure you want to delete YOUR OWN table. 

In [ ]:
# %sql DROP TABLE mysql.default.covid_cases_europe_daily

## Step 1: Explore the List of Data Jobs That Have Been Created on the Cloud

As per the previous example, let's explore the data jobs that have been created on the cloud. This is good practice because it allows you to be aware of what's going on, but also lets you know not to name your data job in a certain way (because it may already exist!).

In [ ]:
! vdk list

## Step 2: Create a Data Job

Now that we have explored the list of created (on the cloud) data jobs, let's create our own.

Keep in mind that we would like to have a sub-folder for the data job,so that our Streamlit script is outside of it and in the main directory.

<font color='red'>**ATTENTION!**</font>

Based on the information above, try creating a data job titled as follows:
* tracking-covid, dash (-)
* your last name, dash (-)
* your favorite sports team, dash (-)
* your favorite drink.

Please do not use team names and numbers that may be parts of any of your passwords, as the data job names will be visible to all. For example, you can create a data job titled "covid-tracking-smith-man-united-cola".

You can chose any team name that you want, but please create the job at the home directory. This will create a sub-folder for the data job. The home directory is /home/jovyan.

Here's an example code:
```
! vdk create -n tracking-covid-avramov-man-united-cola -t amld -p /home/jovyan
```

<font color='green'>**GOOD JOB!**</font>

You can run the command below to see if your data job will show up. It should! In addition, a folder with your data job's name will show up on the left side of your screen, in the directories section.

In [ ]:
! vdk list

## Step 3: Work Out the Data Job Template

As you already know, when you create a data job, VDK automatically downloads some template scripts and files, so that you can get accustomed to the data job's structure. They are super helpful in getting you ready to run your own data jobs. However, let's go ahead and delete these for our example, since we won't be starting from scratch, but please check them out! Alternatively, you can explore the 'vdk create --no-template' option, if you do not want these templates downloaded. Let's go ahead and delete the following files:

- The SQL script: we won't need it.
- The sample Python script: we already have some Python scripts ready for you to edit - more on that below.
- README.md: We already have a README for the entire example, so we can get rid of this.
- requirements.txt: Each data job would need this file if the data job relies on external libraries that VDK does not have. In our case, MyBinder installed those libraries upon startup, so we won't be needing this either.

<font color='red'>**ATTENTION!**</font> 

As such, please run the code below to delete them.

Please change '!!! ENTER DATA JOB NAME HERE' to the name of your data job.

In [ ]:
! rm "!!! ENTER DATA JOB NAME HERE/10_sql_step.sql"
! rm "!!! ENTER DATA JOB NAME HERE/20_python_step.py"
! rm "!!! ENTER DATA JOB NAME HERE/README.md"
! rm "!!! ENTER DATA JOB NAME HERE/requirements.txt"
! rm "!!! ENTER DATA JOB NAME HERE/config.ini"

<font color='green'>**GOOD JOB!**</font>

Now that you have created a data job and gotten rid of the templates, please go inside the subfolder and set up the structure of your data job. Here's the general idea.

We want the data job to have six scripts:

* Let's have one SQL script that creates the Covid cases data table in our cloud DB.
* Let's have one SQL script that creates the Covid deaths data table in our cloud DB.
* Let's have one SQL script that creates the Covid cases and deaths clean table in our cloud DB.
* Let's have one Python script that creates an API call for the Covid cases data and ingests it into our cloud DB.
* Let's have another Python script that creates an API call for the Covid deaths data and ingests it into our cloud DB.
* Let's have one Python script that reads both sets of data from the cloud DB, joins them, cleans them, and saves them in a new table in the DB.
* Let's also have a config.ini file, which specifies how often the data job will be executed, using cron scheduling.

Each of these scripts are present in the sample scripts subfolder. However, we've added some coding challenges inside of them to make things fun!

<font color='red'>**ATTENTION!**</font>

Let's move the sample scripts to the data job subfolder. Please run the code cell below, making sure to enter your data job's name instead of the '!!! ENTER NAME HERE'.

In [ ]:
! mv "sample scripts/01_create_covid_cases_europe_daily.sql" ~/!!! ENTER NAME HERE
! mv "sample scripts/02_create_covid_deaths_europe_daily.sql" ~/!!! ENTER NAME HERE
! mv "sample scripts/03_create_covid_cases_deaths_europe_daily.sql" ~/!!! ENTER NAME HERE
! mv "sample scripts/10_ingest_covid_cases_data.py"  ~/!!! ENTER NAME HERE
! mv "sample scripts/20_ingest_covid_deaths_data.py"  ~/!!! ENTER NAME HERE
! mv "sample scripts/30_clean_merge_transform.py"  ~/!!! ENTER NAME HERE
! mv "sample scripts/config.ini"  ~/!!! ENTER NAME HERE

Great! Now you're all set up with the data job:

* You have created a data job on the cloud.
* You have deleted the template files that you do not need.
* You have moved the sample scripts we provided to the data job sub-folder.

The next step is to begin working on each script in the data job! Let's do it!

## Step 4: Data Job - Define the Covid Cases Table (01_create_covid_cases_europe_daily.sql)

Before we can begin getting data from the API, we need to first create an empty table in our cloud DB that will house the data. We will begin with an SQL script that creates the Covid cases table.

Please open up 01_create_covid_cases_europe_daily.sql in your data job's subfolder. Inside, you will see that the code is not fully populated. We want YOU to do that!

<font color='red'>**ATTENTION!**</font>

Please note that the table name is currently not entered. Instead, it is "!!! ENTER TABLE NAME HERE". We want you to change that so that the table name is called "covid_cases_europe_daily", underscore, your last name, underscore, your favorite sports team, underscore, and your favorite drink. For example: covid_cases_europe_daily_smith_basel_cola.

<font color='green'>**GOOD JOB!**</font>

Nice! You've created the needed table. Let's move on to the next one!

## Step 5: Data Job - Define the Covid Deaths Table (02_create_covid_deaths_europe_daily.sql)

Please open up 02_create_covid_deaths_europe_daily.sql in your data job's subfolder. Inside, you will see that the code is not fully populated. We want YOU to do that!

<font color='red'>**ATTENTION!**</font>

Please note that the table name is currently not entered. Instead, it is "!!! ENTER TABLE NAME HERE". We want you to change that so that the table name is called "covid_deaths_europe_daily", underscore, your last name, underscore, your favorite sports team, underscore, and your favorite drink. For example: covid_deaths_europe_daily_smith_basel_cola.

In addition, please change the column types to column types that make sense. In other words, country and and obs_date should be VARCHAR, but what should the other be? Hint: Look at the previous script!

<font color='green'>**GOOD JOB!**</font>

Nice! You've created the needed table. Let's move on to the next one!

## Step 6: Data Job - Define the Covid Cases and Deaths Clean Table (03_create_clean_full_table.sql)

Please open up 03_create_clean_full_table.sql in your data job's subfolder. Inside, you will see that the code is not fully populated. We want YOU to do that!

<font color='red'>**ATTENTION!**</font>

Please find all of the "!!! ENTER...." instances and make the needed changes, as you have done in the previous two scripts above. The table name should be something like "covid_cases_deaths_europe_daily_smith_cola".

<font color='green'>**GOOD JOB!**</font>

Nice! You've now created all of the needed tables. Let's get ready to ingest some data!

## Step 7: Data Job - Incrementally Ingest Covid Cases Data (10_ingest_covid_cases_data.py)

Please open up 10_ingest_covid_cases_data.py. This script will do an API call to ingest the Covid cases data.

As you are already quite familiar with the way VDK works within Python scripts, the beginning of the script should look familiar. It imports the needed libraries and begins the "run" function. 

From the earlier example, you are already familiar with incremental ingestion. Thus, lines 21 to 30 will look familiar. What you are doing is using job properties so that the script knows how much and what data to ingest. 

## Step 8: Data Job - Incrementally Ingest Covid Deaths Data  (20_ingest_covid_deaths_data.py)

## Step 9: Data Job - Incrementally Build Covid Cases and Deaths Clean Data (30_clean_merge_transform.py)

## Step 10: Deploy the Job to the Cloud

In [ ]:
! vdk deploy -n tracking-covid-avramov-man-united-boza -r "Initial deploy" -p /home/jovyan/tracking-covid-avramov-man-united-boza

In [ ]:
! vdk deploy --show -t amld -n tracking-covid-avramov-man-united-boza

In [ ]:
! vdk list

## Step 11: Building an Interactive Streamlit Dashboard (build_streamlit_dashboard.py)

In [ ]:
! mv "sample scripts/build_streamlit_dashboard.py"  ~

In [ ]:
! streamlit run build_streamlit_dashboard.py
# add "/proxy/8501/" to the URL to go to the Streamlit Web App
# This will block the page until you click Interrupt the Kernel